Running this notebook will read in demo shark data and then create a widget that displays information and mapping locations for a specific shark.

In [ ]:
!pip install pandas ipyleaflet -q

In [ ]:
import pandas as pd
import ipywidgets as widgets
from ipyleaflet import Map, Marker, MarkerCluster

df = pd.read_csv('data/sharks_cleaned.csv')
df.shape

In [ ]:
counts = df.name.value_counts().to_dict()
names = {'' : ''}
for name, count in counts.items():
    visible_name = '%s (%s)' % (name, count)
    names[visible_name] = name
    
choices = widgets.Dropdown(options=names)
info = widgets.HTML()
m = Map()
m.shark_layer = None

def on_select(event):
    ### filter df data
    name = event.owner.value
    matches = df[df.name == name]
    ### Pull out basic info
    info.value = ''
    info.value += 'Name: %s<br/>' % name
    info.value += 'Gender: %s<br/>' % matches.iloc[0].gender
    info.value += 'N Hits: %s<br/>' % len(matches)
    info.value += 'First/last seen: %s - %s' % (matches.datetime.min(), matches.datetime.max())
    ### render map
    if m.shark_layer is not None:
        m.remove_layer(m.shark_layer)
        
    m.center = (matches.latitude.mean(), matches.longitude.mean())
    markers = []
    for row in matches.itertuples():
        marker = Marker(location=(row.latitude, row.longitude), draggable=False)
        markers.append(marker)
    cluster = MarkerCluster(markers=markers)
    m.add_layer(cluster)
    m.shark_layer = cluster
      
    
choices.observe(on_select, names='value')

box = widgets.VBox(children=[choices, info, m])
box